In [1]:
import csv
import codecs
from tqdm import tqdm
from urllib.request import urlretrieve

In [2]:
rows = []
with codecs.open('../data/HomologyTAPE/pdb_chain_scop_uniprot.tsv', encoding='utf-8') as f:
    for row in csv.DictReader(f, skipinitialspace=True, delimiter='\t'):
        rows.append(row)
print(len(rows))
print(rows[0])

104840
{'PDB': '101m', 'CHAIN': 'A', 'SP_PRIMARY': 'P02185', 'SUNID': '15125', 'SCOP_ID': 'd101ma_'}


In [3]:
fold_dic = {}
for row in rows:
    fold_dic[row['SCOP_ID']] = {}
    fold_dic[row['SCOP_ID']]['PDB-chain'] = row['PDB'].upper() + '-' + row['CHAIN']
    fold_dic[row['SCOP_ID']]['uniprot'] = row['SP_PRIMARY']

In [4]:
fold_dic['d101ma_']

{'PDB-chain': '101M-A', 'uniprot': 'P02185'}

In [5]:
with open('../data/HomologyTAPE/test_fold.txt') as f:
    test_file_lines = f.readlines()
test_lines = {}
test_pdb = {}
failed_test_scop = []
for line in test_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    test_lines[data[0]] = data[-1]
    try:
        if fold_dic[data[0]]['PDB-chain'] not in test_pdb.keys():
            test_pdb[fold_dic[data[0]]['PDB-chain']] = []
        test_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_test_scop.append(data[0])

In [6]:
with open('../data/HomologyTAPE/training.txt') as f:
    train_file_lines = f.readlines()
train_lines = {}
train_pdb = {}
failed_train_scop = []
for line in train_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    train_lines[data[0]] = data[-1]
    try:
        if fold_dic[data[0]]['PDB-chain'] not in train_pdb.keys():
            train_pdb[fold_dic[data[0]]['PDB-chain']] = []
        train_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_train_scop.append(data[0])

In [7]:
with open('../data/HomologyTAPE/validation.txt') as f:
    val_file_lines = f.readlines()
val_lines = {}
val_pdb = {}
failed_val_scop = []
for line in val_file_lines:
    data = line.split('\t')
    data = [d.strip() for d in data]
    val_lines[data[0]] = data[-1]
    try:
        if fold_dic[data[0]]['PDB-chain'] not in val_pdb.keys():
            val_pdb[fold_dic[data[0]]['PDB-chain']] = []
        val_pdb[fold_dic[data[0]]['PDB-chain']].append(data[-1])
    except:
        failed_val_scop.append(data[0])

In [8]:
fold_dic['d1ysya1']

{'PDB-chain': '1YSY-A', 'uniprot': 'P0C6X7'}

In [9]:
for key, value in test_pdb.items():
    test_pdb[key] = list(set(value))
for key, value in train_pdb.items():
    test_pdb[key] = list(set(value))
for key, value in val_pdb.items():
    test_pdb[key] = list(set(value))

In [10]:
print(len(test_file_lines))
print(len(train_file_lines))
print(len(val_file_lines))
print(len(failed_test_scop))
print(len(failed_train_scop))
print(len(failed_val_scop))

718
12312
736
17
550
11


In [11]:
all_pdb = list(test_pdb.keys()) + list(train_pdb.keys()) + list(val_pdb.keys())

In [12]:
all_pdb = [pdb[:4] for pdb in all_pdb]
print(len(all_pdb))

21643


In [13]:
import json
no_download = []
with open('../data/HomologyTAPE/all_download.json') as f:
    download_pdb = json.load(f)

In [14]:
for pdb in tqdm(all_pdb):
    if pdb not in download_pdb:
        no_download.append(pdb)

  2%|▏         | 391/21643 [00:01<01:39, 213.53it/s]

100%|██████████| 21643/21643 [01:09<00:00, 310.64it/s]


In [15]:
pdb_chains = list(test_pdb.keys()) + list(train_pdb.keys()) + list(val_pdb.keys())

In [16]:
with open('../data/EnzymeCommission/nrPDB-EC_annot_copy.tsv') as f:
    ec_lines = f.readlines()
with open('../data/nrPDB-GO_annot.tsv') as f:
    go_lines = f.readlines()
download_chains = []
download_ec_chains = [ec.split('\t')[0] for ec in ec_lines[1:]]
download_go_chains = [ec.split('\t')[0] for ec in go_lines[12:]]
download_chains = download_ec_chains + download_go_chains

FileNotFoundError: [Errno 2] No such file or directory: '../data/nrPDB-EC_annot_copy.tsv'

In [ ]:
download_chains = list(set(download_chains))
need_download = []
for pdb in tqdm(pdb_chains):
    if pdb not in download_chains or pdb[:4] not in download_pdb:
        need_download.append(pdb)

100%|██████████| 21643/21643 [03:06<00:00, 116.02it/s]


In [69]:
need_download = [pdb[:4] for pdb in need_download]

['1KF6-B',
 '2BS2-B',
 '1NEK-B',
 '1X9B-A',
 '1HCU-A',
 '1X9D-A',
 '2RI9-A',
 '1DL2-A',
 '1NXC-A',
 '1GXM-A',
 '1HB6-A',
 '1YVR-A',
 '1N4K-A',
 '2AJA-A',
 '1L5J-A',
 '2A9U-A',
 '1QSA-A',
 '1WY6-A',
 '1J1J-A',
 '1LSH-A',
 '1HFE-S',
 '1VQO-2',
 '2V7Q-I',
 '1SA0-E',
 '2F2A-C',
 '1WMI-B',
 '1WIJ-A',
 '2DK4-A',
 '2HJQ-A',
 '2OUT-A',
 '1GYZ-A',
 '1A6Q-A',
 '2CA5-A',
 '2G0U-A',
 '2A26-A',
 '1YSM-A',
 '1Z0P-A',
 '1K4T-A',
 '1TJL-A',
 '1X4T-A',
 '1EIY-A',
 '2IY5-A',
 '1URF-A',
 '1CXZ-B',
 '1E52-A',
 '1QOJ-A',
 '2GSV-A',
 '2GBO-A',
 '1Q0G-A',
 '1T6U-A',
 '2J9U-A',
 '1V74-B',
 '1GM5-A',
 '1SZI-A',
 '3FAP-B',
 '1XZP-A',
 '1GS9-A',
 '2P61-A',
 '1OQC-A',
 '1JR8-A',
 '1YAR-O',
 '1AVO-B',
 '2CHO-A',
 '2CBI-A',
 '2G38-A',
 '2G38-B',
 '2GTS-A',
 '2EIA-A',
 '1EOQ-A',
 '1A8O-A',
 '1D1D-A',
 '2FI2-A',
 '1Y7Q-A',
 '2VLQ-A',
 '2FEF-A',
 '2FUG-1',
 '2HGK-A',
 '2HI7-B',
 '1PI1-A',
 '1R3B-A',
 '2QSB-A',
 '2QZG-A',
 '2BL7-A',
 '2BL8-A',
 '2RLD-A',
 '1ZKE-A',
 '1SKV-A',
 '2AZ0-A',
 '2BZB-A',
 '1PZQ-A',
 '1B0N-A',

In [ ]:
failed = 0
root_dir = '../data/HomologyTAPE/'
idx = 0
download_pdb = []
failed_pdb = []
for pdbid in tqdm(need_download):
    
    # if idx == 10:
    #     break
    try:
        url = 'https://ftp.wwpdb.org/pub/pdb/data/structures/divided/pdb/'+pdbid[1:3]+'/pdb' + pdbid + '.ent.gz'
        # print(url)
        urlretrieve(url, root_dir +'all/' + pdbid.upper() + '.pdb.gz')
        download_pdb.append(pdbid.lower())
    except:
        print("cannot download:", pdbid)
        failed += 1
        failed_pdb.append(pdbid.lower())
    idx += 1
with open('../data/HomologyTAPE/download_pdb.json', 'w') as f:
    json.dump(download_pdb, f)
with open('../data/HomologyTAPE/failed_pdb.json', 'w') as f:
    json.dump(failed_pdb, f)

In [10]:
import h5py

In [15]:
f = h5py.File("../data/HomologyTAPE/d2gyco1.hdf5","r")
print(f.keys())

<KeysViewHDF5 ['amino_chains', 'amino_neighs', 'amino_neighs_hb', 'amino_neighs_sindices', 'amino_neighs_sindices_hb', 'amino_pos', 'amino_types', 'atom_amino_id', 'atom_chain_ids', 'atom_chain_names', 'atom_names', 'atom_pos', 'atom_residue_id', 'atom_residue_names', 'atom_types', 'cov_bond_list', 'cov_bond_list_hb', 'cov_bond_list_sindices', 'cov_bond_list_sindices_hb', 'pos_center']>


In [22]:
f['amino_chains'][()]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=int64)